In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
from matplotlib import pyplot as plt

from hopfield.iterate import hopfield_history, construct_energy_matrix, annealing_curve
from hopfield.energy import energy
from hopfield.plot import make_tracks_3d
from segment.candidate import gen_seg_layered
from segment.track import gen_seg_track_layered
from metrics.tracks import gen_perfect_act
from datasets import get_hits

In [ ]:
event = get_hits('spdsim', 1)
event

In [ ]:
config = {
    'alpha': 1.5,  # forks and joins
    'beta': 0,  # total activation
    'bias': 0,  # activation bias, instead of total activation matrix
    'cosine_power': 5,
    'cosine_min_rewarded': 0.45,
    'cosine_min_allowed': 0.45,
    'distance_power': 0,
    'gamma': 1,
    'threshold': 0.5,  # activation threshold for segment classification
    'tmax': 10,
    'tmin': 0.5,
    'starting_act': 0.2,
    'anneal_steps': 60,
    'total_steps': 100,
    'dropout': 0,
    'learning_rate': 0.2
}

In [ ]:
pos = event[['x', 'y', 'z']].to_numpy()
seg = gen_seg_layered(event)
energy_matrix, crossing_part, curvature_part = construct_energy_matrix(config, pos, seg)
temp_curve = annealing_curve(config['tmin'], config['tmax'], config['anneal_steps'],
                             config['total_steps'] - config['anneal_steps'])
acts = hopfield_history(config, energy_matrix, temp_curve, seg)

In [ ]:
track_segments = gen_seg_track_layered(event)
len(seg), len(track_segments)

In [ ]:
perfect_act = gen_perfect_act(event, seg)
perfect_act

In [ ]:
tcdf = pd.DataFrame({'temp': temp_curve})
tcdf.index.name = 'step'
tcdf.plot()

In [ ]:
energy_history = pd.DataFrame({
    'energy': [energy(energy_matrix, act) for act in acts],
    'curvature_energy': [energy(curvature_part, act) for act in acts],
    'crossing_energy': [energy(crossing_part, act) for act in acts]
})
energy_history.index.name = 'step'
energy_history.energy = 1 + energy_history.energy - energy_history.energy.min()
energy_history.plot(logy=True)
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from metrics.segments import gen_perfect_act

perfect_act = gen_perfect_act(event, seg)
small_history = pd.DataFrame([
    (
        precision_score(perfect_act, act > config['threshold'], zero_division=0),
        recall_score(perfect_act, act > config['threshold'], zero_division=0),
        f1_score(perfect_act, act > config['threshold'], zero_division=0),
        roc_auc_score(perfect_act, act),
        average_precision_score(perfect_act, act),
        act.mean(),
        ((act - perfect_act) ** 2).mean()
    ) for act in acts],
    columns=['precision', 'recall', 'f1', 'roc_auc', 'ap', 'mean_act', 'dist_perfect'])
small_history.index.name = 'step'
small_history.plot()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score

act = acts[-1]
average_precision = average_precision_score(perfect_act, act)
precision, recall, _ = precision_recall_curve(perfect_act, act)
PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision,
                       estimator_name='hopfield').plot()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay

fpr, tpr, _ = roc_curve(perfect_act, act)
RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc_score(perfect_act, act), estimator_name='hopfield').plot()
plt.show()

In [ ]:
from sklearn.metrics import det_curve, DetCurveDisplay

fpr, fnr, _ = det_curve(perfect_act, act)
DetCurveDisplay(fpr=fpr, fnr=fnr).plot()
plt.show()

In [ ]:
n_steps = min(16, len(acts))
steps = np.linspace(0, len(acts) - 1, n_steps, dtype=int)

tracks_3d = []
tracks_projection = []
tracks_by_track = []
for i in steps:
    tp, fp, tn, fn = make_tracks_3d(pos, seg, acts[i], perfect_act, config['threshold'])
    vdims = ['act', 'perfect_act', 'positive', 'true']
    xyz = hv.Overlay([
        hv.Scatter3D(event[event.track == -1], kdims=['x', 'y', 'z'], vdims=['track', 'layer'], label='noise',
                     group='hits'),
        hv.Scatter3D(event[event.track != -1], kdims=['x', 'y', 'z'], vdims=['track', 'layer'], label='hits',
                     group='hits'),
        hv.Path3D(tp, vdims=vdims, label='tp', group='tracks'),
        hv.Path3D(fp, vdims=vdims, label='fp', group='tracks'),
        hv.Path3D(fn, vdims=vdims, label='fn', group='tracks')
    ])
    tracks_3d.append(xyz)

    projection = hv.Points(event[event.track != -1], kdims=['x', 'y'], vdims=['track', 'layer'], label='hits',
                           group='hits') * hv.Points(event[event.track == -1], kdims=['x', 'y'],
                                                     vdims=['track', 'layer'], label='noise', group='hits') * hv.Path(
        tp, kdims=['x', 'y'], vdims=vdims, label='tp', group='tracks') * hv.Path(fp, kdims=['x', 'y'], vdims=vdims,
                                                                                 label='fp', group='tracks') * hv.Path(
        fn, kdims=['x', 'y'], vdims=vdims, label='fn', group='tracks')
    tracks_projection.append(projection)

    nodes = hv.Nodes(event, kdims=['track', 'layer', 'index'])
    no_tn = np.logical_or(acts[i] > config['threshold'], perfect_act > config['threshold'])
    graph = hv.Graph(((*seg[no_tn].transpose(), acts[i][no_tn], perfect_act[no_tn]), nodes),
                     vdims=['act', 'perfect_act'])
    tracks_by_track.append(hv.Overlay([graph]))


In [ ]:
hv.extension('matplotlib')
tracks_3d[-1].opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='brown'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', fig_size=400),
)

In [ ]:
hv.extension('bokeh')
track_history = hv.HoloMap(
    {s: hv.Histogram(np.histogram(acts[s], bins=32, range=(0, 1))) for s in
     np.linspace(0, len(acts) - 1, min(32, len(acts)), dtype=int)},
    kdims='step', group="Activation", label="All") + hv.HoloMap(
    {s: hv.Histogram(np.histogram(acts[s][perfect_act > config['threshold']], bins=32, range=(0, 1))) for s in
     np.linspace(0, len(acts) - 1, min(32, len(acts)), dtype=int)},
    kdims='step', group="Activation", label="On track") + hv.HoloMap(
    {s: hv.Histogram(np.histogram(acts[s][perfect_act < config['threshold']], bins=32, range=(0, 1))) for s in
     np.linspace(0, len(acts) - 1, min(32, len(acts)), dtype=int)},
    kdims='step', group="Activation", label="Off track")

track_history.opts(
    opts.Histogram(width=400, height=400)
)

In [ ]:
hv.extension('plotly')
tracks_3d[-1].opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='brown'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=800, height=800)
)

In [ ]:
hv.extension('matplotlib')
track_history = hv.NdLayout(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(c='black'),
    opts.Scatter3D('Hits.Noise', c='brown'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right'),
    opts.NdLayout(fig_size=200)
).cols(4)

In [ ]:
hv.extension('plotly')
track_history = hv.HoloMap(
    {s: tracks_3d[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Scatter3D(size=2, color='black'),
    opts.Scatter3D('Hits.Noise', color='brown'),
    opts.Path3D(color='black'),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='cyan'),
    opts.Overlay(width=700, height=700)
)
# hv.save(track_history, 'track_history_3d.html', fmt='widgets')

In [ ]:
hv.extension('matplotlib')
track_history = hv.HoloMap(
    {s: tracks_projection[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Points(color='black'),
    opts.Points('Hits.Noise', color='brown'),
    opts.Path(color='black', show_legend=True),
    opts.Path('Tracks.fp', color='red'),
    opts.Path('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', fig_size=256),
)

In [ ]:
hv.extension('bokeh')
track_history = hv.HoloMap(
    {s: tracks_projection[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Points(color='black', tools=['hover'], size=4),
    opts.Points('Hits.Noise', tools=['hover'], color='brown'),
    opts.Path(color='black', show_legend=True),
    opts.Path('Tracks.fp', color='red'),
    opts.Path('Tracks.fn', color='cyan'),
    opts.Overlay(legend_position='right', width=700, height=700)
)


In [ ]:
hv.extension('bokeh')
track_history = hv.NdLayout(
    {s: tracks_by_track[i] for i, s in enumerate(steps)},
    kdims='step'
)
track_history.opts(
    opts.Graph(node_size=8, edge_color='act', colorbar=True),
    opts.EdgePaths(colorbar=True)
).cols(5)

In [ ]:
n_steps = min(16, len(acts))
steps = np.linspace(0, len(acts) - 1, n_steps, dtype=int)

tracks_3d = []
tracks_projection = []
tracks_by_track = []
for i in steps:
    tp, fp, tn, fn = make_tracks_3d(pos, seg, acts[i], perfect_act, config['threshold'])
    vdims = ['act', 'perfect_act', 'positive', 'true']
    xyz = hv.Overlay([
        hv.Path3D(tn, vdims=vdims, label='tn', group='tracks'),
        hv.Path3D(tp, vdims=vdims, label='tp', group='tracks'),
        hv.Path3D(fp, vdims=vdims, label='fp', group='tracks'),
        hv.Path3D(fn, vdims=vdims, label='fn', group='tracks'),
        hv.Scatter3D(event[event.track == -1], kdims=['x', 'y', 'z'], label='noise', group='hits'),
        hv.Scatter3D(event[event.track != -1], kdims=['x', 'y', 'z'], label='hits', group='hits')
    ])
    tracks_3d.append(xyz)

    projection = hv.Path(tn, kdims=['x', 'y'], vdims=vdims, label='tn', group='tracks') * hv.Path(tp, kdims=['x', 'y'],
                                                                                                  vdims=vdims,
                                                                                                  label='tp',
                                                                                                  group='tracks') * hv.Path(
        fp, kdims=['x', 'y'], vdims=vdims, label='fp', group='tracks') * hv.Path(fn, kdims=['x', 'y'], vdims=vdims,
                                                                                 label='fn',
                                                                                 group='tracks') * hv.Points(
        event[event.track != -1], kdims=['x', 'y'], label='hits', group='hits') * hv.Points(event[event.track == -1],
                                                                                            kdims=['x', 'y'],
                                                                                            label='noise', group='hits')
    tracks_projection.append(projection)

    nodes = hv.Nodes(event, kdims=['track', 'layer', 'index'])
    no_tn = np.logical_or(acts[i] > config['threshold'], perfect_act > config['threshold'])
    graph = hv.Graph(((*seg[no_tn].transpose(), acts[i][no_tn], perfect_act[no_tn]), nodes),
                     vdims=['act', 'perfect_act'])
    tracks_by_track.append(hv.Overlay([graph]))

hv.extension('matplotlib')
tracks_3d[-1].opts(
    opts.Scatter3D(c='green'),
    opts.Scatter3D('Hits.Noise', c='black'),
    opts.Path3D(color='black', show_legend=True),
    opts.Path3D('Tracks.fp', color='red'),
    opts.Path3D('Tracks.fn', color='orange'),
    opts.Path3D('Tracks.tn', color='black', alpha=0.1),
    opts.Overlay(legend_position='right', fig_size=400),
)